In [ ]:
!pip install transformers
!git clone https://github.com/ALjone/INF368-Final-Project
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text
!pip install -q tf-models-official
import sys
import numpy as np
import pandas as pd
sys.path.append("/content/INF368-Final-Project")
from Bert import Bert
import os

In [ ]:
!python GPT2Tuner.py \
--train_data_path data/train_labeled_5.csv \
--output_name generated_samples_5.txt \
--epochs 10 \
--batch_size 2 \
--device cuda \
--torch_seed 1 \
--numpy_seed 2 \
--random_seed 3 \
--repeat_num 10

In [2]:
!python GPT2Tuner.py \
--train_data_path data/train_labeled_10.csv \
--output_name generated_samples_10.txt \
--epochs 10 \
--batch_size 2 \
--device cuda \
--torch_seed 1 \
--numpy_seed 2 \
--random_seed 3 \
--repeat_num 10

Starting training:

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



avg_train_loss 10.755517369508743
elapsed time for 1 training epoch :  0:01:43
avg_train_loss 2.440124177932739
elapsed time for 1 training epoch :  0:01:35
avg_train_loss 1.9912845328450204
elapsed time for 1 training epoch :  0:01:35
avg_train_loss 1.8234325051307678
elapsed time for 1 training epoch :  0:01:36
avg_train_loss 1.6345902353525161
elapsed time for 1 training epoch :  0:01:35
Generating sentences
Finished generating sentences.


In [6]:
!python GPT2Tuner.py \
--train_data_path data/train_labeled_25.csv \
--output_name generated_samples_25.txt \
--epochs 10 \
--batch_size 2 \
--device cuda \
--torch_seed 1 \
--numpy_seed 2 \
--random_seed 3 \
--repeat_num 10

Starting training:

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1487 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Traceback (most recent call last):
  File "GPT2Tuner.py", line 142, in <module>
    tuner.save_sentences(int(args.samples_per_class), path=args.output_dir + "/" + args.output_name)
  File "GPT2Tuner.py", line 113, in save_sentences
    f.write(self.tokenizer.decode(sample_output, skip_special_tokens=True).replace("\n", "")+"\n")
  File "D:\Anaconda\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\x96' in position 396: character maps to <undefined>



avg_train_loss 5.252679090499878
elapsed time for 1 training epoch :  0:10:09
avg_train_loss 1.7242548483610154
elapsed time for 1 training epoch :  0:10:03
avg_train_loss 1.6241241866350173
elapsed time for 1 training epoch :  0:09:44
avg_train_loss 1.470165878534317
elapsed time for 1 training epoch :  0:10:09
avg_train_loss 1.2941617393493652
elapsed time for 1 training epoch :  0:09:03
avg_train_loss 1.1303855109214782
elapsed time for 1 training epoch :  0:09:02
avg_train_loss 0.9625671485066414
elapsed time for 1 training epoch :  0:09:03
avg_train_loss 0.8146653261780739
elapsed time for 1 training epoch :  0:09:11
avg_train_loss 0.6817608812451362
elapsed time for 1 training epoch :  0:11:03
avg_train_loss 0.5570713968575001
elapsed time for 1 training epoch :  0:16:43
Generating sentences


In [4]:
!python GPT2Tuner.py \
--train_data_path data/train_labeled_50.csv \
--output_name generated_samples_50.txt \
--epochs 10 \
--batch_size 2 \
--device cuda \
--torch_seed 1 \
--numpy_seed 2 \
--random_seed 3 \
--repeat_num 10

Starting training:
avg_train_loss 3.7483344689011573
elapsed time for 1 training epoch :  0:12:29
avg_train_loss 1.6732652413845062
elapsed time for 1 training epoch :  0:14:36
avg_train_loss 1.2771680292487144
elapsed time for 1 training epoch :  0:19:47
avg_train_loss 1.023910652846098
elapsed time for 1 training epoch :  0:27:10
avg_train_loss 0.8440950420498848
elapsed time for 1 training epoch :  0:49:47
Generating sentences
Finished generating sentences.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1487 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
batch_size = 4
seed = 0
learning_rate = 5e-5
epochs=5


In [ ]:
 def get_sentences(train_path, synthesized_path, sentences_per_label):
        h = Bert(num_classes = 2, random_state = seed)
        train = pd.read_csv(train_path)
        h.train(train.text, train.label,learning_rate=learning_rate,batch_size=batch_size,epochs=epochs) 

        with open(synthesized_path, "r") as file:
            sentences = file.readlines()
        
        #TODO get the X best of each label, not just top X best total

        #TODO also gather data in dict before putting it in dataframe
        #https://stackoverflow.com/questions/57000903/what-is-the-fastest-and-most-efficient-way-to-append-rows-to-a-dataframe

        labels = []
        cleaned_sentences = []
        for sentence in sentences:
            #sentence_parts = sentence.split(maxsplit = 1)
            labels.append(sentence[:3])
            cleaned_sentences.append(sentence[3:])

        df = pd.DataFrame(columns=["text", "label", "predicted label", "confidence"])
        for label, sentence in zip(labels, cleaned_sentences):
            predictions = h.predict_label_proba([sentence])[0]
            new_row = {"text": sentence, "label": label, "predicted label" : predictions[0], "confidence" : predictions[1]}
            df = df.append(new_row, ignore_index=True)
        
        candidates = df.loc[df["label"] == df["predicted label"]]
        candidates.sort_values("confidence", ascending=False, inplace=True)
        #Make sure we have enough labels
        if candidates.size >= sentences_per_label*len(set(labels)):
            return candidates.head(sentences_per_label*len(set(labels)))[["text", "label", "confidence"]]
        else:
            print("Not enough correct labels synthesized, returning what little we have")
            return candidates[["text", "label", "confidence"]]

In [ ]:
for datapoints in [5, 10]:
  get_sentences("/content/INF368-Final-Project/data/train_labeled_"+str(datapoints) + ".csv",
                "/content/INF368-Final-Project/data/generated_samples_" + str(datapoints)+".txt", 10).to_csv("/content/INF368-Final-Project/data/filtered_data_" + str(datapoints)+".csv")


In [ ]:
seed = 42
data = pd.DataFrame(columns=["n_per_class", "accuracy"])
for datapoints in [5, 10]:
  h = Bert(num_classes = 2, random_state = seed)
  train = pd.read_csv("/content/INF368-Final-Project/data/filtered_data_" + str(datapoints)+".csv")
  h.train(train.text, train.label,learning_rate=learning_rate,batch_size=batch_size,epochs=epochs)  
  performance = h.evaluate_from_path("/content/INF368-Final-Project/data/test.csv")[1]
  row = {"n_per_class" : datapoints, "accuracy": performance}
  data = data.append(row, ignore_index=True)
data

if not os.path.exists('results'):
      os.mkdir('results')
data.to_csv("results/LAMBADA_results.csv")